In [1]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import tabulate
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# pip install transformers==4.45.2 



d:\code\dl-hw5\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset('dair-ai/emotion')
print(dataset)

labels = dataset["train"].features["label"].names
print(labels)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})
['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']


In [3]:
model_name = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels)
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
def tokenize_data(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

# Токенизация всех данных
tokenized_dataset = dataset.map(tokenize_data, batched=True)
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map: 100%|██████████| 2000/2000 [00:00<00:00, 9433.51 examples/s]


In [5]:
def compute_metrics(eval_pred):
    logits, true_labels = eval_pred  # Переименовали переменную
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(true_labels, predictions),
        **classification_report(
            true_labels, 
            predictions,
            target_names=labels,  # Используем глобальный список названий
            output_dict=True,
            zero_division=0
        )["macro avg"]
    }


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=1000,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    max_steps=1000,
    logging_steps=100,
    weight_decay=0.01,
    metric_for_best_model="f1-score",
    logging_dir="./logs",
    report_to="none",
    load_best_model_at_end=True,
    save_total_limit=2,
    # lr_scheduler_type="cosine",
    # warmup_steps=500
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics
)

results = trainer.evaluate(tokenized_dataset["test"])
print(tabulate.tabulate(
    results.items(),
    headers=["Метрика", "Значение"],
    tablefmt="grid",
    floatfmt=".4f"
))
trainer.train()

results = trainer.evaluate(tokenized_dataset["test"])
print(tabulate.tabulate(
    results.items(),
    headers=["Метрика", "Значение"],
    tablefmt="grid",
    floatfmt=".4f"
))


Test metrics: {'eval_loss': 0.7932651042938232, 'eval_model_preparation_time': 0.0, 'eval_accuracy': 0.7425, 'eval_precision': 0.7402512345382611, 'eval_recall': 0.5454750466210642, 'eval_f1-score': 0.5292401848522555, 'eval_support': 2000.0, 'eval_runtime': 30.2114, 'eval_samples_per_second': 66.2, 'eval_steps_per_second': 4.138}
+-----------------------------+------------+
| Метрика                     |   Значение |
+=============================+============+
| eval_loss                   |     0.7933 |
+-----------------------------+------------+
| eval_model_preparation_time |     0.0000 |
+-----------------------------+------------+
| eval_accuracy               |     0.7425 |
+-----------------------------+------------+
| eval_precision              |     0.7403 |
+-----------------------------+------------+
| eval_recall                 |     0.5455 |
+-----------------------------+------------+
| eval_f1-score               |     0.5292 |
+-----------------------------+------

Step,Training Loss,Validation Loss,Model Preparation Time,Accuracy,Precision,Recall,F1-score,Support
100,0.481300,0.602014,0.000000,0.798500,0.795935,0.695663,0.717404,2000.000000
200,0.556300,0.401258,0.000000,0.879500,0.863867,0.819401,0.836907,2000.000000
300,0.412000,0.370869,0.000000,0.887500,0.885845,0.833811,0.848792,2000.000000
400,0.345200,0.298690,0.000000,0.904000,0.886308,0.881649,0.879431,2000.000000
500,0.295300,0.253372,0.000000,0.914500,0.880174,0.895599,0.887184,2000.000000
600,0.268200,0.242581,0.000000,0.914500,0.890440,0.884771,0.885680,2000.000000
700,0.236800,0.229441,0.000000,0.923500,0.904171,0.898934,0.900193,2000.000000
800,0.233000,0.210620,0.000000,0.922500,0.893911,0.899958,0.896235,2000.000000
900,0.224700,0.195859,0.000000,0.926500,0.894704,0.911191,0.902439,2000.000000
1000,0.229800,0.194467,0.000000,0.925000,0.892626,0.914553,0.902922,2000.000000


Test metrics: {'eval_loss': 0.19975651800632477, 'eval_model_preparation_time': 0.0, 'eval_accuracy': 0.9255, 'eval_precision': 0.8758080764820901, 'eval_recall': 0.9015332050069241, 'eval_f1-score': 0.8875884063063505, 'eval_support': 2000.0, 'eval_runtime': 22.5506, 'eval_samples_per_second': 88.689, 'eval_steps_per_second': 5.543, 'epoch': 1.0}
+-----------------------------+------------+
| Метрика                     |   Значение |
+=============================+============+
| eval_loss                   |     0.1998 |
+-----------------------------+------------+
| eval_model_preparation_time |     0.0000 |
+-----------------------------+------------+
| eval_accuracy               |     0.9255 |
+-----------------------------+------------+
| eval_precision              |     0.8758 |
+-----------------------------+------------+
| eval_recall                 |     0.9015 |
+-----------------------------+------------+
| eval_f1-score               |     0.8876 |
+-------------------

In [15]:
from torch import nn
from transformers import AutoModel

# Кастомная классификационная голова
class CustomClassifier(nn.Module):
    def __init__(self, hidden_size, num_labels, dropout_prob=0.1):
        super().__init__()
        self.dense = nn.Linear(hidden_size, 1024)
        self.dropout = nn.Dropout(dropout_prob)
        self.out_proj = nn.Linear(1024, num_labels)
        self.relu = nn.ReLU()

    def forward(self, pooled_output):
        x = self.dense(pooled_output)  # [batch_size, 256]
        x = self.relu(x)
        x = self.dropout(x)
        return self.out_proj(x)        # [batch_size, num_labels]

# Модифицируем модель BERT
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels)
)

# Замораживаем все слои BERT
for param in model.parameters():
    param.requires_grad = False

# Добавляем кастомную голову
hidden_size = model.config.hidden_size  # 768 для bert-base
num_labels = len(labels)                          # 6 эмоций в датасете
model.classifier = CustomClassifier(hidden_size, num_labels)

# Проверяем, что обучаются только параметры классификатора
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Обучается: {name}")

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=1000,
    learning_rate=1e-4, # Больше lr, так как обучаем только голову
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    max_steps=10000,
    logging_steps=200,
    weight_decay=0.01,
    metric_for_best_model="f1-score",
    logging_dir="./logs",
    report_to="none",
    load_best_model_at_end=True,
    save_total_limit=2,
    # lr_scheduler_type="cosine",
    # warmup_steps=500
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics
)

trainer.train()
results = trainer.evaluate(tokenized_dataset["test"])
print(tabulate.tabulate(
    results.items(),
    headers=["Метрика", "Значение"],
    tablefmt="grid",
    floatfmt=".4f"
))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Обучается: classifier.dense.weight
Обучается: classifier.dense.bias
Обучается: classifier.out_proj.weight
Обучается: classifier.out_proj.bias


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1-score,Support
200,1.580600,1.566283,0.392000,0.130832,0.197472,0.149039,2000.000000
400,1.545700,1.558187,0.441500,0.172626,0.236536,0.194022,2000.000000
600,1.516600,1.556525,0.412000,0.151174,0.204422,0.150668,2000.000000
800,1.526300,1.508552,0.457500,0.157347,0.245720,0.188290,2000.000000
1000,1.500700,1.478144,0.463000,0.159239,0.248456,0.190569,2000.000000
1200,1.480000,1.465558,0.467500,0.161749,0.251250,0.192665,2000.000000
1400,1.471300,1.455693,0.475500,0.160201,0.253182,0.194655,2000.000000
1600,1.448000,1.436043,0.486500,0.326724,0.255758,0.198223,2000.000000
1800,1.474200,1.414599,0.491000,0.217141,0.257642,0.199582,2000.000000
2000,1.452200,1.413316,0.493500,0.222652,0.262595,0.207293,2000.000000


Test metrics: {'eval_loss': 1.381722092628479, 'eval_accuracy': 0.486, 'eval_precision': 0.27523213925002504, 'eval_recall': 0.25627310887950566, 'eval_f1-score': 0.20800118134501563, 'eval_support': 2000.0, 'eval_runtime': 22.5217, 'eval_samples_per_second': 88.803, 'eval_steps_per_second': 5.55, 'epoch': 10.0}
+-------------------------+------------+
| Метрика                 |   Значение |
+=========================+============+
| eval_loss               |     1.3817 |
+-------------------------+------------+
| eval_accuracy           |     0.4860 |
+-------------------------+------------+
| eval_precision          |     0.2752 |
+-------------------------+------------+
| eval_recall             |     0.2563 |
+-------------------------+------------+
| eval_f1-score           |     0.2080 |
+-------------------------+------------+
| eval_support            |  2000.0000 |
+-------------------------+------------+
| eval_runtime            |    22.5217 |
+-------------------------+---

In [12]:
from peft import (
    PromptTuningConfig,
    get_peft_model,
    TaskType
)
from transformers import AutoModelForSequenceClassification

# 1. Конфигурация Prompt Tuning
peft_config = PromptTuningConfig(
    task_type=TaskType.SEQ_CLS,
    num_virtual_tokens=20,  # Длина мягкого промпта
    token_dim=768,          # Размерность эмбеддингов BERT
    # num_transformer_submodules=1,  # Для encoder-only моделей
    prompt_tuning_init="TEXT",
    prompt_tuning_init_text="Classify the emotion in the text:",
    base_model_name_or_path="bert-base-uncased",
    tokenizer_name_or_path="bert-base-uncased"
)

# 2. Загрузка модели
model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-uncased",
    num_labels=len(labels),
    return_dict=True
)

# 3. Обертка модели в PEFT
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()  # Обучаемые параметры: ~0.1%

# 4. Заморозка основной модели (опционально)
# for param in model.base_model.parameters():
#     param.requires_grad = False

training_args = TrainingArguments(
    output_dir="./peft_results",
    learning_rate=3e-4,           # Выше обычного для промптов
    per_device_train_batch_size=32,
    num_train_epochs=5,          # Нужно больше эпох
    logging_steps=100,
    save_strategy="no"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics
)

trainer.train()
results = trainer.evaluate(tokenized_dataset["test"])
print(tabulate.tabulate(
    results.items(),
    headers=["Метрика", "Значение"],
    tablefmt="grid",
    floatfmt=".4f"
))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 15,360 || all params: 109,502,214 || trainable%: 0.0140


  4%|▍         | 100/2500 [01:33<32:01,  1.25it/s]

{'loss': 1.8187, 'grad_norm': 0.17120012640953064, 'learning_rate': 0.00028799999999999995, 'epoch': 0.2}


  8%|▊         | 200/2500 [02:54<30:46,  1.25it/s]

...

{'loss': 1.5756, 'grad_norm': 1.0910353660583496, 'learning_rate': 1.1999999999999999e-05, 'epoch': 4.8}


100%|██████████| 2500/2500 [35:04<00:00,  1.19it/s]


{'loss': 1.5852, 'grad_norm': 0.8338690996170044, 'learning_rate': 0.0, 'epoch': 5.0}
{'train_runtime': 2104.5312, 'train_samples_per_second': 38.013, 'train_steps_per_second': 1.188, 'train_loss': 1.6135260314941406, 'epoch': 5.0}


100%|██████████| 250/250 [00:25<00:00,  9.70it/s]

+-------------------------+------------+
| Метрика                 |   Значение |
+=========================+============+
| eval_loss               |     1.5444 |
+-------------------------+------------+
| eval_accuracy           |     0.4125 |
+-------------------------+------------+
| eval_precision          |     0.1381 |
+-------------------------+------------+
| eval_recall             |     0.2071 |
+-------------------------+------------+
| eval_f1-score           |     0.1564 |
+-------------------------+------------+
| eval_support            |  2000.0000 |
+-------------------------+------------+
| eval_runtime            |    25.8858 |
+-------------------------+------------+
| eval_samples_per_second |    77.2620 |
+-------------------------+------------+
| eval_steps_per_second   |     9.6580 |
+-------------------------+------------+
| epoch                   |     5.0000 |
+-------------------------+------------+


In [9]:

from peft import (
    LoraConfig, 
    TaskType, 
    get_peft_model
)
from transformers import AutoModelForSequenceClassification

# 1. Конфигурация LoRA
lora_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        inference_mode=False,
        r=8,                # Ранг адаптеров
        lora_alpha=16,      # Коэффициент масштабирования
        lora_dropout=0.1,   # Дропаут для регуляризации
        target_modules=["query", "value"],  # Слои для применения LoRA
        bias="none"
    )

# 2. Загрузка модели
model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-uncased",
    num_labels=6
)


# Создаем новую конфиг
peft_config = lora_config

# Обертка модели
lora_model = get_peft_model(model, peft_config)
lora_model.print_trainable_parameters()

# Обучение
training_args = TrainingArguments(
    output_dir=f"./lora_results",
    learning_rate=3e-4,
    per_device_train_batch_size=32,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    logging_steps=200
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics
)

trainer.train()
results = trainer.evaluate(tokenized_dataset["test"])
print(tabulate.tabulate(
    results.items(),
    headers=["Метрика", "Значение"],
    tablefmt="grid",
    floatfmt=".4f"
))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 299,526 || all params: 109,786,380 || trainable%: 0.2728


  8%|▊         | 200/2500 [02:48<31:26,  1.22it/s]

{'loss': 1.349, 'grad_norm': 2.0876481533050537, 'learning_rate': 0.000276, 'epoch': 0.4}


 16%|█▌        | 400/2500 [05:33<28:45,  1.22it/s]

...

{'loss': 0.1844, 'grad_norm': 2.3966174125671387, 'learning_rate': 1.1999999999999999e-05, 'epoch': 4.8}


100%|██████████| 2500/2500 [35:20<00:00,  1.28it/s]





















































































































































































































































                                                   

100%|██████████| 2500/2500 [35:48<00:00,  1.28it/s]


100%|██████████| 2500/2500 [35:48<00:00,  1.16it/s]


{'eval_loss': 0.1654692143201828, 'eval_accuracy': 0.938, 'eval_precision': 0.9193392246932405, 'eval_recall': 0.9178323360827396, 'eval_f1-score': 0.9184753633939998, 'eval_support': 2000.0, 'eval_runtime': 27.0808, 'eval_samples_per_second': 73.853, 'eval_steps_per_second': 9.232, 'epoch': 5.0}
{'train_runtime': 2148.5027, 'train_samples_per_second': 37.235, 'train_steps_per_second': 1.164, 'train_loss': 0.40780635681152344, 'epoch': 5.0}


100%|██████████| 250/250 [00:26<00:00,  9.35it/s]

+-------------------------+------------+
| Метрика                 |   Значение |
+=========================+============+
| eval_loss               |     0.1978 |
+-------------------------+------------+
| eval_accuracy           |     0.9220 |
+-------------------------+------------+
| eval_precision          |     0.8764 |
+-------------------------+------------+
| eval_recall             |     0.8873 |
+-------------------------+------------+
| eval_f1-score           |     0.8813 |
+-------------------------+------------+
| eval_support            |  2000.0000 |
+-------------------------+------------+
| eval_runtime            |    26.8544 |
+-------------------------+------------+
| eval_samples_per_second |    74.4760 |
+-------------------------+------------+
| eval_steps_per_second   |     9.3090 |
+-------------------------+------------+
| epoch                   |     5.0000 |
+-------------------------+------------+
